##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

### Answer Q1
- For a fair comparison, the Custom CNN was trained for 3 epochs and achieved 50.18% accuracy. 
- The MobileNetV2 (fine-tuned) model achieved 85.37% accuracy.
- The ResNet50V2 (fine-tuned) model achieved 91.62% accuracy.

Therefore, ResNet50V2 achieved the highest classification accuracy among all tested models.

### Answer Q2
Custom cnn was the fastest

### Answer Q3
ResNet50V2 achieved the highest accuracy because it is a deep network with residual (skip) connections, which improve gradient flow and allow better feature learning.

MobileNetV2 is designed to be lightweight and efficient, using depthwise separable convolutions, which reduce parameters but also reduce model capacity.

The Custom CNN performed much lower because it was trained from scratch without pretrained ImageNet features, and it has a simpler architecture compared to the deep pretrained models.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# -----------------------------
# 1) Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers (SparseCategoricalCrossentropy)
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

# -----------------------------
# 2) Data augmentation
# -----------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

# -----------------------------
# 3) Build MobileNetV2 backbone
# -----------------------------
mobilenet_base = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

mobilenet_base.trainable = False   # Freeze first (feature extractor)

# -----------------------------
# 4) Full Model
# -----------------------------
mobilenet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224),
    layers.Lambda(preprocess_input),
    mobilenet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)  # logits
], name="cifar10_mobilenetv2")

mobilenet_model.summary()

# -----------------------------
# 5) Compile + Train (Frozen)
# -----------------------------
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

# -----------------------------
# 6) Test / Evaluate
# -----------------------------
test_loss, test_acc_frozen = mobilenet_model.evaluate(x_test, y_test, verbose=0)
print("MobileNetV2 (frozen) test accuracy:", test_acc_frozen)
print("MobileNetV2(frozen) test loss    :", test_loss)


/opt/anaconda3/envs/cv_lab/lib/python3.13/site-packages/keras/src/datasets/cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


Model: "cifar10_mobilenetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 253s 357ms/step - accuracy: 0.6818 - loss: 0.9117 - val_accuracy: 0.8044 - val_loss: 0.5619
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 288s 408ms/step - accuracy: 0.7502 - loss: 0.7219 - val_accuracy: 0.8226 - val_loss: 0.5066
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 293s 417ms/step - accuracy: 0.7589 - loss: 0.6915 - val_accuracy: 0.8404 - val_loss: 0.4708
MobileNetV2 (frozen) test accuracy: 0.8278999924659729
MobileNetV2(frozen) test loss    : 0.49519383907318115


In [2]:
# -----------------------------
# 7) Fine-tune Last Layers
# -----------------------------
mobilenet_base.trainable = True

for layer in mobilenet_base.layers[:-30]:
    layer.trainable = False

print("Trainable layers:",
      sum(l.trainable for l in mobilenet_base.layers),
      "/", len(mobilenet_base.layers))

mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

test_loss_ft, test_acc_ft = mobilenet_model.evaluate(x_test, y_test, verbose=0)
print("MobileNetV2 (fine-tuned) test accuracy:", test_acc_ft)
print("MobileNetV2 (fine-tuned) test loss    :", test_loss_ft)

Trainable layers: 30 / 154
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 416s 587ms/step - accuracy: 0.7127 - loss: 0.8325 - val_accuracy: 0.8326 - val_loss: 0.4933
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 424s 603ms/step - accuracy: 0.7779 - loss: 0.6407 - val_accuracy: 0.8456 - val_loss: 0.4438
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 425s 604ms/step - accuracy: 0.8014 - loss: 0.5732 - val_accuracy: 0.8554 - val_loss: 0.4115
MobileNetV2 (fine-tuned) test accuracy: 0.8537999987602234
MobileNetV2 (fine-tuned) test loss    : 0.4286457598209381


In [ ]:
# CUSTOM CNN WITH 3 EPOCH FOR COMPARISON
# -----------------------------
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -----------------------------
# 1) Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers (SparseCategoricalCrossentropy)
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

# -----------------------------
# 2) Data augmentation 
# -----------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

# -----------------------------
# 3) Build Custom CNN
# -----------------------------
custom_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Rescaling(1./255),   # normalize [0,1]

    layers.Conv2D(32, (3,3), padding="same", activation="relu"),
    layers.Conv2D(32, (3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), padding="same", activation="relu"),
    layers.Conv2D(64, (3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), padding="same", activation="relu"),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.30),

    layers.Dense(10)  # logits
], name="cifar10_custom_cnn")

custom_model.summary()

# -----------------------------
# 4) Compile + Train (3 epochs)
# -----------------------------
custom_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history_cnn = custom_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,         
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# -----------------------------
# 5) Test / Evaluate
# -----------------------------
test_loss_cnn, test_acc_cnn = custom_model.evaluate(x_test, y_test, verbose=0)
print("Custom CNN (3 epochs) test accuracy:", test_acc_cnn)
print("Custom CNN (3 epochs) test loss    :", test_loss_cnn)


/opt/anaconda3/envs/cv_lab/lib/python3.13/site-packages/keras/src/datasets/cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


Model: "cifar10_custom_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,714 (549.66 KB)

 Trainable params: 140,714 (549.66 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 31s 43ms/step - accuracy: 0.3095 - loss: 1.8206 - val_accuracy: 0.4316 - val_loss: 1.5607 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 31s 43ms/step - accuracy: 0.4452 - loss: 1.5010 - val_accuracy: 0.5054 - val_loss: 1.3552 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 32s 45ms/step - accuracy: 0.4962 - loss: 1.3671 - val_accuracy: 0.5176 - val_loss: 1.3991 - learning_rate: 0.0010
Custom CNN (3 epochs) test accuracy: 0.501800000667572
Custom CNN (3 epochs) test loss    : 1.414796233177185
